In [ ]:
import pandas as pd
from omegaconf import OmegaConf
from tqdm import tqdm

from model import Model
from dataset.crowspairs_dataset import CrowsPairsDataset

In [ ]:

CROWSPAIRS_DATASET_PATH = "path/to/crows_pairs.csv" # set path to crows_pairs.csv

### Load the Dataset

In [ ]:
dataset_args = OmegaConf.create({
    "shuffle": True,                    # whether to shuffle the dataset
    "seed": 42,                         # seed for shuffling
    # "num_samples": 5,                   # number of samples to load (for debugging)
    "source": CROWSPAIRS_DATASET_PATH,  # path to the dataset
    "category_column": "bias_type"      # column in the dataset that contains the category
})


dataset = CrowsPairsDataset(dataset_args)
dataset.load_dataset()
dataset = dataset.subsets
dataset.head()

### Compute Log Probabilities

In [ ]:
# create model
model_args = OmegaConf.create({
    "model_name": "llama3chat",                                         # name of the model (llam2chat, mistralchat, llama3chat)
    "deployment": {"method": "quantization", "type": "bitsandbytes", "nbits": 4},      # deployment dict, can be None, method: "pruning" (type: "wanda_unstruct", "wanda_struct") or "quantization" (type: "awq", "bitsandbytes", "kvcachequant" with nbits "4" or "8")
    "device": "cuda",                                                   # device to run the model on
    "sampling_method": "greedy"                                         # sampling method for the model (greedy, sampling)
})

model = Model(model_args)

### Get Log Probabilities

In [ ]:
def process_sample(sample: dict) -> dict:
    # combine all input fields into a single prompt
    prompt_more = sample["sent_more"]
    prompt_less = sample["sent_less"]

    output_more = model.predict(prompt_more)[1]
    output_less = model.predict(prompt_less)[1]

    return output_more, output_less


log_prob_more = []
log_prob_less = []

for idx in tqdm(range(len(dataset))):
    processed_sample = process_sample(dataset.iloc[idx])
    log_prob_more.append(processed_sample[0])
    log_prob_less.append(processed_sample[1])

dataset["sent_more_log_prob"] = log_prob_more
dataset["sent_less_log_prob"] = log_prob_less

dataset.head()


### Compute Bias Scores

In [ ]:
def compute_crowspairs_scores(sent_more_log_probs: pd.Series, sent_less_log_probs: pd.Series):
    """
    Compute the log probability difference between the more and less stereotypical sentences in the CrowsPairs dataset.
    Args:
        sent_more_log_probs: log probability scores for the more stereotypical sentence
        sent_less_log_probs: log probability scores for the less stereotypical sentence
    Returns:
        log_prob_diff: log probability difference between the more and less stereotypical sentences
        prefers_more: boolean indicating whether the model prefers the more stereotypical sentence
    """
    log_prob_diff = sent_more_log_probs - sent_less_log_probs
    prefers_more = log_prob_diff > 0
    return log_prob_diff, prefers_more

dataset["log_prob_diff"], dataset["prefers_more"] = compute_crowspairs_scores(dataset["sent_more_log_prob"], dataset["sent_less_log_prob"])
dataset.head()

In [ ]:
dataset_logprobdiff = dataset["log_prob_diff"].mean()
dataset_prefersmore = dataset["prefers_more"].mean() * 100

print(f"Average log probability difference: {dataset_logprobdiff:.4f}")
print(f"Model prefers more stereotypical sentence: {dataset_prefersmore:.2f}%")

In [ ]:
categories = dataset[dataset_args.category_column].unique()
category_scores = {
    str(name): round(dataset[dataset[dataset_args.category_column] == name]["prefers_more"].mean(),2) for name in categories
}

print("Prefers-More scores per category:")
for name, score in category_scores.items():
    print(f"{name}: {score}")